## Blackjack Problem
In this problem we'll be working with a simplified version of [blackjack](https://en.wikipedia.org/wiki/Blackjack) (aka twenty-one). In this version there is one player (who you'll control) and a dealer. Play proceeds as follows:

- The player is dealt two face-up cards. The dealer is dealt one face-up card.
- The player may ask to be dealt another card ('hit') as many times as they wish. If the sum of their cards exceeds 21, they lose the round immediately.
- The dealer then deals additional cards to himself until either:
    - the sum of the dealer's cards exceeds 21, in which case the player wins the round
    - the sum of the dealer's cards is greater than or equal to 17. If the player's total is greater than the dealer's, the player wins. Otherwise, the dealer wins (even in case of a tie).
    
When calculating the sum of cards, Jack, Queen, and King count for 10. Aces can count as 1 or 11 (when referring to a player's "total" above, we mean the largest total that can be made without exceeding 21. So e.g. A+8 = 19, A+8+8 = 17)

For this problem, you'll write a function representing the player's decision-making strategy in this game. We've provided a very unintelligent implementation below:

In [4]:
import numpy as np
import itertools, random

In [21]:
class Dealer:
    def __init__(self):
        self.deck = list(itertools.product(range(1,14),['Spade','Heart','Diamond','Club']))
        self.cards = []
        self.nhigh_aces = 0
        self.nlow_aces = 0
        self.total_points = 0
    
    def new_game(self, players):
        self.cards = []
        self.nhigh_aces = 0
        self.nlow_aces = 0
        self.total_points = 0
        if type(players) == list:
            for p in players:
                if has
        self.shuffle_cards()
    
    def shuffle_cards(self):
        # shuffle the cards
        if not self.cards:
            random.shuffle(self.deck)
    
    def deal(self, to_whom=None):
        # deal one card at a time
        if hasattr(to_whom, 'cards'):
            to_whom.cards.append(self.deck.pop(0))
        else:
            self.cards.append(self.deck.pop(0))
    
    def count_total_points(self):
        point_list = [c[0] if c[0] < 10 else 10 for c in dealer.cards]
        other_points = np.sum([p for p in point_list if p > 1])
        naces = len(point_list) - len(other_points)
        if naces <= 0:
            self.nhigh_aces = 0
            self.nlow_aces = 0
            self.total_points = other_points
        else:
            self.nhigh_aces = 1 if other_points + naces + 10 <= 21 else 0
            self.nlow_aces = naces - self.nhigh_aces
            self.total_points = other_points + self.nlow_aces + self.nhigh_aces * 11
    
    def should_hit(self):
        # if the dealer should hit the next card, return true
        self.count_total_points()
        if self.total_points >= 17:
            return False
        else:
            return True

In [20]:
dealer = Dealer()
dealer.shuffle_cards()
for i in range(5):
    dealer.deal()

17